In [7]:
import os
import azure.cognitiveservices.speech as speechsdk
from dotenv import load_dotenv

os.environ['SPEECH_KEY'] = 'ef888ca2899e4e40844b94ca568e5bff'
os.environ['SPEECH_REGION'] = 'northcentralus'

def recognize_from_audio_file(audio_filename):
    # 
    speech_key = os.environ.get('SPEECH_KEY')
    speech_region = os.environ.get('SPEECH_REGION')

    # 
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.speech_recognition_language="en-US"

    # 
    audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)

    # 
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print(f"Recognizing speech from audio file: {audio_filename}")
    speech_recognition_result = speech_recognizer.recognize_once_async().get()

    # 
    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(speech_recognition_result.text))
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized.")
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))

audio_filename = 'C:/Users/hechu/Downloads/1st_Meeting_4th_Session_Open-ended_Working_Group_on_Reducing_Space_Threats-Kaltura-1_vyc9qx1u-hls-audio-English.mp4'
recognize_from_audio_file(audio_filename)

RuntimeError: Exception with error code: 
[CALL STACK BEGIN]

    > pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - recognizer_create_speech_recognizer_from_config
    - recognizer_create_speech_recognizer_from_config

[CALL STACK END]

Exception with an error code: 0xa (SPXERR_INVALID_HEADER)

In [ ]:
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst
import azure.cognitiveservices.speech as speechsdk
import time

# 初始化 GStreamer
Gst.init(None)

class GStreamerAudioStream(speechsdk.audio.PullAudioInputStreamCallback):
    def __init__(self, mp4_filename):
        super().__init__()
        self.pipeline = Gst.parse_launch(f'filesrc location="{mp4_filename}" ! decodebin ! audioconvert ! audioresample ! appsink name=sink sync=true')
        self.appsink = self.pipeline.get_by_name('sink')
        self.pipeline.set_state(Gst.State.PLAYING)

    def read(self, buffer):
        sample = self.appsink.emit('pull-sample')
        if sample:
            buf = sample.get_buffer()
            data = buf.extract_dup(0, buf.get_size())
            buffer[:len(data)] = data
            return len(data)
        return 0

    def close(self):
        self.pipeline.set_state(Gst.State.NULL)

def recognize_from_mp4(mp4_filename):
    speech_key = "your_speech_key"
    speech_region = "your_speech_region"
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    
    stream = GStreamerAudioStream(mp4_filename)
    audio_config = speechsdk.audio.AudioConfig(stream=stream)
    recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    # 设置识别器的各种事件
    recognizer.recognizing.connect(lambda evt: print(f"RECOGNIZING: {evt.result.text}"))
    recognizer.recognized.connect(lambda evt: print(f"RECOGNIZED: {evt.result.text}"))
    recognizer.session_started.connect(lambda evt: print("SESSION STARTED: {}".format(evt)))
    recognizer.session_stopped.connect(lambda evt: print("SESSION STOPPED"))
    recognizer.canceled.connect(lambda evt: print(f"CANCELED: {evt}"))

    # 开始连续识别
    recognizer.start_continuous_recognition()
    try:
        while True:
            time.sleep(0.5)
    except KeyboardInterrupt:
        # 停止连续识别
        recognizer.stop_continuous_recognition()

mp4_filename = 'path_to_your_mp4_file.mp4'
recognize_from_mp4(mp4_filename)